In [3]:

# import some Python dependencies

#bring in relevant libraries
import pandas as pd
from pandas import ExcelWriter
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import re
import requests
import datetime
import time
import httplib, urllib, json, locale
from urlparse import urlparse
import operator
import bson
from collections import OrderedDict
from pymongo import MongoClient
from bson.objectid import ObjectId
import csv
import json
import urllib2
from instagram import InstagramAPI
import sys
import re

In [4]:
merchant_ids = []
top_posts = []
df_list= []
#connecting to our db
connection = MongoClient(host = '')
db=connection.heroku_ckbhgr9b
merchant = db['Merchants']
ACCESS_TOKEN = ''
for merchants in merchant.find():
     merchant_ids.append(str(merchants['_id']))
        
        
        




        
for MERCHANT_ID in merchant_ids:
    #eventually we'll want this to be brought in externally from mongo

    #object_id = 'ObjectId(\"' + MERCHANT_ID + '\")'
    doc1 = []

    for doc in merchant.find({'_id':ObjectId(MERCHANT_ID)}):
         doc1.append(doc)
    try:
        access_token = doc1[0]['pos_access_token']

        record=db['Reports']

        access_token = str(access_token)
    except:
        pass

#access_token = ''

#the below functions are described in detail here: http://facebook-sdk.readthedocs.io/en/latest/api.html
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)

            print "Error for URL %s: %s" % (url, datetime.datetime.now())

            print "Error for URL %s: %s" % (url, datetime.datetime.now())
    return response.read()

def getFacebookPageFeedData(page_id, access_token, num_statuses):

    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(20).summary(true),comments.limit(20).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + parameters

    # retrieve data
    data = json.loads(request_until_succeed(url))

    return data

def processFacebookPageFeedStatus(status):

    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.

    # Additionally, some items may not always exist,
    # so must check for existence first

    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.

    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    


    # Nested items require chaining dictionary keys.
    comment_content = []
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    for i in range(len(status["comments"]["data"])):
        comment_content.append(status['comments']['data'][i]['message'])
    #for i in len(num_comments):
        #comment_content.append('' if 'comments' not in status.keys else status['comments'])
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']


    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares, comment_content)


for merchants in merchant.find():
     merchant_ids.append(str(merchants['_id']))

for MERCHANT_ID in merchant_ids:
    #eventually we'll want this to be brought in externally from mongo
    record=db['Marketing']

    for doc in merchant.find({'_id':ObjectId(MERCHANT_ID)}):            

        try:
            #if this part doesn't work, they likely don't have facebook setup. Therefore we use try except
            page_id = str(doc['facebook_page_id'])
            m_id = str(doc['merchant_id'])
            name = str(doc['name'])
            #access_token = str(doc['facebook_token'])
            statuses = getFacebookPageFeedData(page_id, ACCESS_TOKEN, 30)['data']
            #print statuses
            comments = []
            for comment in statuses:
                #print comment
                comments.append(processFacebookPageFeedStatus(comment))
            
            
            df = pd.DataFrame(comments)
            #setting column names
            df.columns = ['post_id','message','type_of_post','content_category','link_associated','date','likes','comment_quant','shares', 'comment_content']
            #sorting based on likes first, then comments
            df = df.sort_values(['likes','comment_quant'], ascending=False).reset_index(drop=True)
            #isolating only the columns we want
            df_top_post_att = df[['message','content_category','link_associated', 'likes', 'comment_quant','comment_content']].head(5)
            #taking only the top 5 entries
            df_list.append(df_top_post_att)
            #making a dictionary out of this
            dict_top_posts = df_top_post_att.to_dict('records')
            #append to our top posts list, just in case we need a list of all top posts from every client
            top_posts.append(dict_top_posts)
            #upload data to merchant collection of mongo
            record.update_many({
              'merchant_uid': MERCHANT_ID
                },{
                '$set': {
                    'name':name,
                    'id':MERCHANT_ID,
                    'facebook_page_id':page_id,
                    'merchant_id':m_id,
                    'top_5_facebook_posts': dict_top_posts,
                    'fb_business_account_id':'?',
                    'instagram_acct_id':'?',
                    'instgram_bus_acct_id':'?'
                      }
                }, upsert=True)



        

#insta account id
#insta business account id
#top 5 instagram post data [{{post text, post image/video, likes, comments[]}}]
            
            
        except:
            pass

        

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


comment_content_dict